<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2023TeamB/blob/main/Importierung%20Externer%20Tabellen%20in%20die%20SQL%20Quelldatenbank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Ggf. fehlende Pakete  installieren
!pip install ipython-sql
!pip install sqlalchemy
!pip install petl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 KB 10.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for petl: filename=petl-1.7.12-py3-none-any.whl size=226970 sha256=f070d2e0f55e13f435e3988c11cb2c82d73e92ebc5b5ff135bba02247c01c0b7
  Stored in directory: /root/.cache/pip/wheels/b0/20/a5/96748ad301276597fbd3df013975828fe5978edc4587f79514
Successfully built petl


In [2]:
from google.colab import drive
import sqlalchemy as sqla
import pandas as pd
import petl as etl
import csv
import sys
%load_ext sql

In [3]:
# Google Drive einbinden
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
base_url_staging = "/content/gdrive/MyDrive/Gruppe_B/DB_File"


In [5]:
#db_path_staging        =  "/content/gdrive/MyDrive/Gruppe_B/DB_File/staging.db"
#db_path_source_covid  = base_url_staging + "source_covid19_test.db"
db_path_source_covid  ="/content/gdrive/MyDrive/Gruppe_B/DB_File/source_covid19_test.db"


#db_url_staging        = "sqlite:///" + db_path_staging
db_url_source_covid  = "sqlite:///" + db_path_source_covid


#db_staging            = sqla.create_engine(db_url_staging)
db_source_covid         = sqla.create_engine(db_url_source_covid)


#%sql $db_url_staging
%sql $db_url_source_covid


'Connected: @/content/gdrive/MyDrive/Gruppe_B/DB_File/source_covid19_test.db'

In [6]:
%%sql $db_url_source_covid
ATTACH DATABASE '{db_path_source_covid}' AS covid;


Done.


[]

In [25]:
import sqlite3

conn = sqlite3.connect(db_path_source_covid)
cursor = conn.cursor()

cursor.execute("SELECT name from sqlite_master where type='table';")
tables = cursor.fetchall()

for table in tables:
    print(table[0])

conn.close()


disease
patients
encounters
careplans
conditions
medications
procedures
observations
devices
imaging_studies
immunizations
grading
immunizations_classification
medications_classification


In [13]:
import sqlite3

conn = sqlite3.connect(db_path_source_covid)
cursor = conn.cursor()

cursor.execute("DROP TABLE d_patient")
conn.commit()

conn.close()

In [15]:
import pandas as pd

df = pd.read_csv('/content/gdrive/MyDrive/Gruppe_B/SNOMED CT Codes und Grading Datenmanagement.xlsx - symptom and procedure data.csv')

conn = sqlite3.connect(db_path_source_covid)
df.to_sql('table_name', conn, if_exists='replace', index=False)

conn.close()

In [26]:
import sqlite3

conn = sqlite3.connect(db_path_source_covid)
cursor = conn.cursor()

cursor.execute("SELECT * FROM grading LIMIT 5")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

(234466008.0, 'Acquired coagulation disorder (disorder)', 'conditions', 3.0)
(132281000119108.0, 'Acute deep venous thrombosis (disorder)', 'conditions', 3.0)
(706870000.0, 'Acute pulmonary embolism (disorder)', 'conditions', 4.0)
(67782005.0, 'Acute respiratory distress syndrome (disorder)', 'conditions', 5.0)
(65710008.0, 'Acute respiratory failure (disorder)', 'conditions', 5.0)


In [18]:
import sqlite3

conn = sqlite3.connect(db_path_source_covid)
cursor = conn.cursor()

cursor.execute("ALTER TABLE table_name RENAME TO grading")
conn.commit()

conn.close()

In [20]:
import pandas as pd

df = pd.read_csv('/content/gdrive/MyDrive/Gruppe_B/immunizations_classification - immunizations_classification.csv')

conn = sqlite3.connect(db_path_source_covid)
df.to_sql('table_name', conn, if_exists='replace', index=False)

conn.close()

In [22]:
import sqlite3

conn = sqlite3.connect(db_path_source_covid)
cursor = conn.cursor()

cursor.execute("ALTER TABLE table_name RENAME TO immunizations_classification")
conn.commit()

conn.close()

In [24]:
import pandas as pd

df = pd.read_csv('/content/gdrive/MyDrive/Gruppe_B/medication_classification - medication_classification.csv')

conn = sqlite3.connect(db_path_source_covid)
df.to_sql('medications_classification', conn, if_exists='replace', index=False)

conn.close()

In [28]:
import os

os.rename(db_path_source_covid, '/content/gdrive/MyDrive/Gruppe_B/Updated_Source_DB/source_covid.db')

db_path_source_covid = '/content/gdrive/MyDrive/Gruppe_B/Updated_Source_DB/source_covid.db'
db_url_source_covid = "sqlite:///" + db_path_source_covid
db_source_covid = sqla.create_engine(db_url_source_covid)

In [29]:
import pandas as pd

table_names = pd.read_sql_query("SELECT name from sqlite_master WHERE type='table';", db_source_covid)
print(table_names)

                            name
0                        disease
1                       patients
2                     encounters
3                      careplans
4                     conditions
5                    medications
6                     procedures
7                   observations
8                        devices
9                imaging_studies
10                 immunizations
11                       grading
12  immunizations_classification
13    medications_classification
